In [1]:
import numpy as np
from lme.lme_forecast_verbose import LME
import lme.utils as utils

#### Simulate data

In [3]:
n_loc = 20
n_age = 8
n_sex = 2
n_year = 10
N = n_loc*n_age*n_sex*n_year
dimensions = [n_loc, n_age, n_sex, n_year]
np.random.seed(127)
X = np.insert(np.random.randn(N,2),0,1,axis=1)
gamma_true = 0.1
u_true = np.random.randn(n_loc)*np.sqrt(gamma_true)
v_true = np.random.randn(n_loc*n_age)*np.sqrt(gamma_true)
beta_true = [1., .5, .5]
cov = np.random.randn(n_age)
cov_dim = [1, n_age, 1, 1]
cov_dim_bool = [False, True, False, False]
Z = np.tile(np.kron(np.identity(n_age),np.ones((n_sex*n_year,1))),(n_loc,1))
Y_true = X.dot(beta_true) + np.kron(np.identity(n_loc),np.ones((n_year*n_sex*n_age,1))).dot(u_true) \
   + np.kron(np.identity(n_loc*n_age),np.ones((n_sex*n_year,1))).dot(v_true)*(Z.dot(cov))
delta_true = .05
Y = Y_true + np.random.randn(N)*np.sqrt(delta_true)
print(Y.shape)
Y2 = X.dot(beta_true) + np.random.randn(N)*np.sqrt(delta_true)
print(np.linalg.solve(np.transpose(X).dot(X),np.transpose(X).dot(Y)))

(3200,)
[1.10618652 0.5135996  0.50696761]


#### Build Model
$$ y = \beta_0 + X\beta + \pi_l + \text{cov}\pi_{l,a} $$

In [5]:
model = LME(dimensions, 1, Y, {'cov1':(X[:,1],[True]*4), 'cov2':(X[:,2], [True]*4), 'cov3':(cov, cov_dim_bool)}, 
            {}, ['cov1','cov2'], global_intercept=True, 
            random_effects={'intercept':[True,False,False,False],'cov3':[True,True,False,False]})

In [6]:
model.optimize()

n_groups 20
k_beta 3
k_gamma 9
total number of fixed effects variables 13
fit with gamma fixed...
finished...elapsed 0.6393532752990723


In [7]:
model.beta_soln

array([1.12947817, 0.50012956, 0.50678926])

In [8]:
model.gamma_soln

array([0.08051535, 0.0972084 , 0.0972084 , 0.0972084 , 0.0972084 ,
       0.0972084 , 0.0972084 , 0.0972084 , 0.0972084 ])

In [9]:
model.delta_soln

array([0.04971155])

In [10]:
model.postVarGlobal()
model.postVarRandom()

In [11]:
beta_samples, u_samples = model.draw()

In [12]:
dataset = utils.saveDraws(beta_samples, u_samples, ['intercept','x1','x2'], [['location'], ['location','age']],
                ['pi_location', 'pi_location_age'])

In [13]:
dataset

<xarray.Dataset>
Dimensions:          (age: 8, cov: 3, draw: 10, location: 20)
Coordinates:
  * location         (location) int64 1 2 3 4 5 6 7 8 ... 14 15 16 17 18 19 20
  * draw             (draw) int64 1 2 3 4 5 6 7 8 9 10
  * age              (age) int64 1 2 3 4 5 6 7 8
  * cov              (cov) <U9 'intercept' 'x1' 'x2'
Data variables:
    pi_location      (location, draw) float64 1.113 1.17 1.218 ... 0.4043 0.3355
    pi_location_age  (location, age, draw) float64 3.343 3.438 ... 2.137 1.907
    beta_global      (cov, draw) float64 1.113 1.058 1.056 ... 0.507 0.5083